![image source: https://www.artbreeder.com/image/5172f176d52800751b4463f4d554](images/2025-01-21-streamlit-deployment-aws.jpg)

## Introduction

I recently embarked on a journey to rapidly prototype a personal LLM application. Like many developers, I was drawn to Streamlit for its simplicity and Python-first approach. The ability to quickly design interactive UIs without wrestling with HTML or CSS made it the perfect fit. However, the road to deploying that Streamlit app on AWS turned out to be much more challenging than I initially anticipated. I naively thought that containerizing my app with Docker would be the golden ticket, and I'd be up and running in no time. I was wrong! This post will take you through my learning experience, detailing the various approaches I tried to host the Streamlit application on the AWS cloud, and share what I learned through the process. From serverless lambda functions to dedicated virtual machines, I'll describe what worked, what didn’t, and most importantly, why. By the end of the post, I hope that my experience helps you avoid the pitfalls and saves you from many days of frustration. Let's begin!

## The Initial Assumption: Docker + Cloud = Easy Deployment

With my Streamlit application working perfectly on my local machine, I confidently assumed the next step would be straightforward. "Dockerize it, and deploy it to the cloud," I thought. After all, Docker simplifies application packaging, and cloud platforms like AWS are built for containerized apps, right? My understanding was that the cloud provided a wide array of services to host containerized apps, automatically managing network configurations, and quickly exposing my app through a public URL. I naively believed it would be a seamless and fast process.

To understand where I went wrong, it's helpful to understand the internal architecture of Streamlit. As shown in the diagram below, a Streamlit application consists of three main parts: a React-based frontend, a Tornado web server, and the Streamlit library. The React frontend is responsible for rendering the user interface, while the Tornado web server manages the application's logic and communication with the frontend. What's crucial is that the frontend and the Tornado web server communicate through the `WebSocket protocol`, enabling real-time, bi-directional communication. This is a key detail which is easy to overlook when starting out.

![](images/2025-01-21-streamlit-deployment-aws/streamlit-architecture.png){.lightbox}

### Diving Deep: Streamlit Network Communication

To truly grasp the importance of WebSockets for Streamlit, let’s take a look at what’s happening under the hood during its network communication. I have used Wireshark to capture the network communication that happens when a simple Streamlit app (shown below) is loaded in a browser.

![](images/2025-01-21-streamlit-deployment-aws/streamlit-hello-page.png){.lightbox}

As you can see, the app simply displays "Hello From Streamlit" and is running on `localhost:8501`.

The screenshot below shows the Wireshark capture of the complete network communication trace.

![](images/2025-01-21-streamlit-deployment-aws/wireshark-complete-trace.png){.lightbox}

Initially, the communication starts with the standard HTTP protocol, and it can be seen by the green lines. The browser sends GET requests to fetch resources like JavaScript and CSS files. Then, after initial handshakes, the protocol switches to WebSockets which can be seen in the blue lines. The communication over WebSockets is for the actual application logic. This screenshot highlights the critical role WebSockets play in Streamlit’s operation.

Let’s take a closer look. 

**Stage 1: First Contact with Server**

The first request from the browser is a simple GET request for the main page. The server then sends back an HTML document. This is shown in the below image.

![](images/2025-01-21-streamlit-deployment-aws/wireshark-tornado-server.png){.lightbox}

Here, we can see that the server identifies itself as a "TornadoServer" and the content type is `text/html`. This confirms that the underlying web server in Streamlit is indeed Tornado. 

**Stage 2: Initial HTML Payload from Server**

The response body contains the HTML page that needs to be rendered, and the browser then parses it and starts loading the associated CSS and JS files.

![](images/2025-01-21-streamlit-deployment-aws/wireshark-loaded-page.png){.lightbox}

The following code block is the HTML page that is returned by the server.

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

<!--
 Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022-2024)

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
-->

<!DOCTYPE html>
<html lang="en">

<head>
   <meta charset="UTF-8" />
   <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
   <link rel="shortcut icon" href="./favicon.png" />
   <link rel="preload" href="./static/media/SourceSansPro-Regular.DZLUzqI4.woff2" as="font" type="font/woff2"
       crossorigin />
   <link rel="preload" href="./static/media/SourceSansPro-SemiBold.sKQIyTMz.woff2" as="font" type="font/woff2"
       crossorigin />
   <link rel="preload" href="./static/media/SourceSansPro-Bold.-6c9oR8J.woff2" as="font" type="font/woff2"
       crossorigin />

   <title>Streamlit</title>

   <!-- initialize window.prerenderReady to false and then set to true in React app when app is ready for indexing -->
   <script>
       window.prerenderReady = false
   </script>
   <script type="module" crossorigin src="./static/js/index.Phesr84n.js"></script>
   <link rel="stylesheet" crossorigin href="./static/css/index.CcCsR6ah.css">
</head>

<body>
   <noscript>You need to enable JavaScript to run this app.</noscript>
   <div id="root"></div>
</body>

</html>

After loading the HTML page, the embedded JS code starts making subsequent HTTP requests for fetching CSS and JavaScript files. Once these files are loaded, the client is ready to switch to the WebSocket protocol.

**Stage 3: Switching of Protocols**

The next image shows that the browser requests the server to switch from HTTP to WebSocket using the 101 Switching Protocol message.

![](images/2025-01-21-streamlit-deployment-aws/wireshark-switching-protocol.png){.lightbox}

After this handshake, the communication from client and server happens using the WebSocket protocol. The image below shows how the application data is sent from the server to the client using WebSockets.

**Stage 4: Application Logic**

![](images/2025-01-21-streamlit-deployment-aws/wireshark-content.png){.lightbox}

This shows how the data containing the message "Hello From Streamlit" is exchanged through WebSockets. After that, most of the communication between the client and server consists of ping/pong messages, which indicates that the connection is active. This pattern of sending data using WebSockets is what makes Streamlit apps interactive.

## First Attempt: The Serverless Approach with AWS Lambda

Given my initial goal of quick prototyping and a preference for serverless architectures, AWS Lambda seemed like the ideal first choice. It promised a cost-effective and low-maintenance solution where I could just upload my code and let AWS handle the infrastructure. I envisioned deploying the Streamlit app as a Lambda function and then expose it to the public internet through an API Gateway endpoint. My understanding was that this would be the easiest, cheapest, and fastest way to make my app accessible to others.

To make this happen, I used the Lambda Web Adapter, which is designed to allow Lambda functions to respond to HTTP requests like a typical web server. I created a Docker image containing my Streamlit application and configured the Lambda function to use this image. Everything seemed to be in order, and I was excited to see my app live on the cloud. I uploaded my code, configured my endpoint, and confidently navigated to the public URL.

The page loaded, but to my utter frustration, it was just an empty page. No errors, just a blank screen! I refreshed the page, checked my code, and looked at the logs, and everything seemed to be fine. I was able to get a response from the server, indicating that my request was reaching the Lambda function and that my streamlit app was running. My browser was making the connection, and the server was responding as well. However, I was unable to see my streamlit app!

This is when I discovered the critical limitation: Lambda functions and the Lambda Web Adapter, operate using a request-response pattern with the HTTP protocol. However, Streamlit relies heavily on WebSockets for its real-time interactivity and communication. Since HTTP connections are not persistent, the connection was being closed after the initial request, and the web sockets were never able to establish a connection with the server. The initial request was processed, and the page was sent to the client, but no websocket connection was established. This was not a bug, but a fundamental mismatch in how Streamlit and Lambda work. This prompted me to realize that I had to think differently. This was the first major learning moment for me in my journey.